Copyright (c) Microsoft Corporation.
Licensed under the MIT License.

# Quantum-Inspired Optimization

## Binary optimization problems take the general form


$ \displaystyle\text{Minimize: } \sum_{k} term_k = \sum_k c_k \prod_{i} x_i \text{ where } x_i \in \{ 0,1 \} \text{ or } \{ -1 , 1 \} \text{ and } c_k \in \mathbb{R} $

### for example

$ \displaystyle\text{this problem: } 13 + 17x_0 + 23x_1x_3x_{77} $

#### is represented by these Terms in the Python SDK:

In [ ]:
 [ Term ( c = 13.0 , indices = [] ) , Term [ c = 17.0 indices = [ 0 ] ] , Term [ c = 23.0 indices = 1 , 3 , 77 ] ] 

### Note

The inspiration for this scenario is Vincent's blog post [here](https://vincent.frl/quantum-secret-santa/).  

On his blog, Vincent uses [Q# and the QDK](https://docs.microsoft.com/en-us/azure/quantum/overview-what-is-qsharp-and-qdk) to solve this scenario and here we'll use [Quantum-Inspired Optimzation](https://docs.microsoft.com/en-us/azure/quantum/optimization-what-is-quantum-optimization).



## Secret Santa Scenario

Vincent, Tess, and Uma write their names on slips of paper and place them in a jar.  Then, each draws a slip of paper at random.  Each person buys a small gift and writes a poem for the person whose name they have drawn.  If they draw thier own name, they return the slip of paper and re-draw.

To represent this problem, we can use 6 binary variables, as outlined in the table below:

$
\begin{array}[t]{ | l | c | c | c | }
    \hline
    \textit{ - buys -> } & \textbf{Vincent} & \textbf{Tess} & \textbf{Uma} \\ \hline
    \textbf{ Vincent } & \text{ -- } & x_0 & x_1 \\ 
    \textbf{ Tess } & x_2 & \text{ -- } & x_3 \\
    \textbf{ Uma } & x_3 & x_4 & \text{ -- } \\
    \hline
\end{array}
$

The constraints for the problem can be expressed as doing logical ANDs ($ \land $) of variables that are EXLUSIVE-ORd ($ \oplus $) together, like this:

$
( x_0 \oplus x_1 ) \land ( x_2 \oplus x_3 ) \land ( x_4 \oplus x_5 ) \land ( x_2 \oplus x_4 ) \land ( x_0 \oplus x_5 ) \land ( x_1 \oplus x_3 )
$

$
\text{ where } x_i \in \{ 0,1 \} 
$


The truth table for exclusive or $ \oplus $ is below (one variable or the other is $ \textbf{one} $, but not both:

$
\begin{array}[t]{ | c | c | c | }
    \hline
    \textbf x_0 & \textbf x_1 & \textbf x_0 \oplus \textbf x_1 \\ \hline
    0 & 0 & 0 \\ 
    0 & 1 & 1 \\ 
    1 & 0 & 1 \\ 
    1 & 1 & 0 \\ 
    \hline
\end{array}
$


Using the truth table, we can see how the constraints are derrived. Looking at the scenario table in the cell above:
Reading the first $ \textbf{row} $ of the table, Vincent may buy a gift and write a poem for Tess or for Uma, but not both.
Reading the first $ \textbf{column} $ of the table, Vincent may recieve a gift and poem from Tess or from Uma, but not both.

So, how do we represent $ ( x_0 \oplus x_1 ) $ in Terms that our cost function will understand?

Keeping in mind that we want to $ \textbf{minimize} $ our cost function, what if we do the following?

$ ( x_0 + x_1 - 1 )^2 $  Let's check the truth table:

$
\begin{array}[t]{ | c | c | c | }
    \hline
    \textbf x_0 & \textbf x_1 & \textbf ( x_0 + x_1 - 1 )^2 \\ \hline
    0 & 0 & 1 \\ 
    0 & 1 & 0 \\ 
    1 & 0 & 0 \\ 
    1 & 1 & 1 \\ 
    \hline
\end{array}
$

Since we want to minimize the cost function, getting zero for the answers we want is the correct result.  We are almost there, we just need to expand the squared formula--remember [multiplying polynomials](https://en.wikipedia.org/wiki/Polynomial)?

So, the expanded formula that we need to implement is:  $ x_0^2 + x_1^2 + 2x_0x_1 - 2x_0 - 2x_1 + 1 $

We build up the Terms in the helper function $ \textbf{built-terms} $ in the code cell below, but instead of using $ x_0 and x_1 $, we use the indeces for our variables instead (i and j).  So $ x_0 \oplus x_1 $ would be buld_terms( 0 , 1 ).

In [ ]:
from typing import List
from azure.quantum import Workspace
from azure.quantum.optimization import Problem, ProblemType, Term, SimulatedAnnealing 

In [ ]:
def build_terms ( i : int , j : int ) :
    """
    Construct Terms for a row or a column ( two variables ) of the secret santa matrix

    Arguments:
    i (int): index of first variable
    j (int): index of second variable

    """
    
    terms = [ Term ( c = 1.0 , indices = [ i , i ] ) ]        # x(i)^2
    terms.append ( Term ( c = 1.0 , indices = [ j , j ] ) )   # x(j)^2
    terms.append ( Term ( c = 2.0 , indices = [ i , j ] ) )   # 2x(i)x(j) 
    terms.append ( Term ( c = -2.0 , indices = [ i ] ) )      # -2x(i)
    terms.append ( Term ( c = -2.0 , indices = [ j ] ) )      # -2x(j)
    terms.append ( Term ( c = 1.0 , indices = [] ) )          # +1

    return terms

We have one more helper function, which takes the answer returned from the service and interprets it, based on the Scenario Table, above.

In [ ]:
def print_results ( config : dict ) :
    """
    print results of run

    Arguements:
    config (dictionary): config returned from solver
    """
    result =  { '0' : 'Vincent buys Tess a gift and writes her a poem' ,
                '1' : 'Vincent buys Uma a gift and writes her a poem' ,
                '2' : 'Tess buys Vincent a gift and writes him a poem' ,
                '3' : 'Tess buys Uma a gift and writes her a poem' ,
                '4' : 'Uma buys Vincent a gift and writes him a poem' ,
                '5' : 'Uma buys Tess a gift and writes her a poem' }

    for key, val in config.items() :
        if val == 1 :
            print ( result [ key ] )

Now, we get into the core of our logic, which is to sign into our Azure Quantum Workspace, authenticate to the Service and then build our Terms to form a Problem, submit the Problem to the service and then print the results.

In [ ]:
workspace = Workspace(
    subscription_id =   'xxx', # add your subscription_id
    resource_group =    'xxx', # add your resource_group
    name =              'xxx', # add your workspace name
)

workspace.login()
print ( 'logged into workspace' )


"""
build secret santa matrix

        Vincent Tess Uma
Vincent    -    x(0) x(1)
Tess      x(2)   -   x(3)
Uma	      x(4)  x(5)  -
"""

#       row 0                 + row 1                 + row 2                
terms = build_terms ( 0 , 1 ) + build_terms ( 2 , 3 ) + build_terms ( 4 , 5 )

#             + col 0                 + col 1                 + col 2
terms = terms + build_terms ( 2 , 4 ) + build_terms ( 0 , 5 ) + build_terms ( 1 , 3 )


print ( 'terms' )
print ( terms )
print ( ' ' )

problem = Problem ( name = 'secret santa' , problem_type = ProblemType.pubo , terms = terms )

solver = SimulatedAnnealing ( workspace , timeout = 2 )

print ( solver )

print ( 'calling solver' )
result = solver.optimize ( problem )

print ( 'response from solver' )
print ( result )
print ( ' ' )

print_results ( result [ "configuration" ] )